<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Feature Detectors


For this notebook we need to do the install of the contrib modules from OpenCV. Contrib contains patended algorithms and others under development. SIFT and ORB algorithms are in contrib module

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

In [ ]:
# import OpenCV library
import cv2

# we will use the following import to display images in colab:
from google.colab.patches import cv2_imshow

import numpy as np


In [ ]:
!wget https://github.com/opencv/opencv/blob/master/samples/data/rubberwhale1.png?raw=true -O rubberwhale1.png

# read image
img = cv2.imread('rubberwhale1.png', cv2.IMREAD_COLOR)


In [ ]:
cv2_imshow(img)

## Gradient computation

Feature detectors are interested in areas where high gradients are present: lets compute the gradient (find the areas with high frequencies) with the sobel operator that we saw in the previous chapter:

In [ ]:
# lets compute the filter with horizontal and vertical:

# sobel x direction parameters (sx)
s_xorder = 1
s_yorder = 1
s_ksize = 3

img_sobel = cv2.Sobel(img,cv2.CV_64F,s_xorder,s_yorder,ksize=s_ksize) 

In [ ]:
# lets display the edges:
cv2_imshow(img_sobel)

We can see that the borders of objects are the areas with higher gradients: those areas should be the ones containing more feature descriptors!

## Harris detector

OpenCV has the [cornerHarris](https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#gac1fc3598018010880e370e2f709b4345) function to compute the Harris corner detector


In [ ]:

# the input of harris detector is a gray image:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)



cornerHarris function:

```

 cv.cornerHarris(	src, blockSize, ksize, k[, dst[, borderType]]	) ->	dst

Parameters:
  src single-channel 8-bit or floating-point image.
  blockSize	Neighborhood size 
  ksize	Aperture parameter for the Sobel operator.
  k	Harris detector free parameter.

```


In [ ]:
blockSize = 2
ksize = 3
k = 0.04
dst = cv2.cornerHarris(gray,blockSize, ksize, k)


The output of the cornerHarris is a value for each pixel. Higher values indicate higher probability of having a corner / interest point in that location. Lets see the output of the harris detector:


In [ ]:
# Normalizing
dst_norm = np.empty(dst.shape, dtype=np.float32)
cv2.normalize(dst, dst_norm, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
dst_norm_scaled = cv2.convertScaleAbs(dst_norm)

In [ ]:
cv2_imshow(dst_norm_scaled)

The output is similar at the sobel filter but edges where both gradients are high are more prominent (white pixels)

We can find the harris corner points thresholding the output image: 

In [ ]:
# Threshold harris (optimal depends on each image)
threshold_harris = 0.02

# lets make a copy of the input image to plot the results
img_harris = img.copy()

# pixels higher than the threshold provided are changed to red color:
img_harris[dst>threshold_harris*dst.max()]=[0,0,255]

In [ ]:
cv2_imshow(img_harris)

Try with different thresholds and determine which is better for the example image 

## More advanced detectors: SIFT, SURF and ORB

We  will execute the different algorithms in parallel since all the algorithms are in the contib modules and the API is equivalent for both of them

* [SIFT](https://docs.opencv.org/4.5.4/d7/d60/classcv_1_1SIFT.html):  Scale Invariant Feature Transform

* [ORB](https://docs.opencv.org/4.5.4/db/d95/classcv_1_1ORB.html):  oriented BRIEF


We can initialize the algorithms (lets use default parameters):

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
orb = cv2.ORB_create()

We can use the detector in our image:

In [ ]:
keypoints_sift = sift.detect(img)
keypoints_orb = orb.detect(img)


And plot the results:

In [ ]:
img_sift = np.empty((img.shape[0], img.shape[1], 3), dtype=np.uint8)
img_sift = cv2.drawKeypoints(img, keypoints_sift, img_sift)

img_orb = np.empty((img.shape[0], img.shape[1], 3), dtype=np.uint8)
img_orb = cv2.drawKeypoints(img, keypoints_orb, img_orb)

In [ ]:
cv2_imshow(cv2.hconcat([img_sift, img_orb]))

SIFT and ORB detectors obtain different keypoints than harris. Can you change the parameters of SIFT and ORB and see how the keypoints detected change?